In [1]:
from utils import*
import gc
from random import randint
import time
import multiprocessing
import pandas as pd
import random


In [2]:
scales = np.arange(1,24)        # Scales for CWT
csvdir = '[path_to]/csv/SW & SSW CSV Files'     # Directory of CSV Files
edfdir = '[path_to]/edf/Abnormal EDF Files'	  # Directory of Abnormal EDF files
main_path = '[path_to]/scallogramTest/'
normalEdf_dir = '[path_to]/edf/Normal EDF Files/'
data_csv = '[path_to]/data_split.csv'
random.seed(444)

In [3]:
Label_data = np.empty((0,19))
Epochs_data = np.empty((0,19,400))

dest_list = ['Normal', 'Slowing Waves', 'Spike and Sharp waves']  # Names of subfolders with in main folder
waveletsTypes = ['mexh','morl', 'gaus1', 'gaus2']
splits = ['train', 'valid', 'test']

allFileList = os.listdir(csvdir)
allEdfList = os.listdir(normalEdf_dir)


df = pd.read_csv(data_csv)


train_abnormal = df[(df['Class'] == 'abnormal') & (df['Type'] == 'train') & (df['Age'] >= 6)]
train_abnormal = train_abnormal['File'].tolist()

test_abnormal = df[(df['Class'] == 'abnormal') & (df['Type'] == 'test') & (df['Age'] >= 6)]
test_abnormal = test_abnormal['File'].tolist()

valid_abnormal = df[(df['Class'] == 'abnormal') & (df['Type'] == 'valid') & (df['Age'] >= 6)]
valid_abnormal = valid_abnormal['File'].tolist()


train_normal = df[(df['Class'] == 'normal') & (df['Type'] == 'train') & (df['Age'] >= 6)]
train_normal = train_normal['File'].tolist()

test_normal = df[(df['Class'] == 'normal') & (df['Type'] == 'test') & (df['Age'] >= 6)]
test_normal = test_normal['File'].tolist()

valid_normal = df[(df['Class'] == 'normal') & (df['Type'] == 'valid') & (df['Age'] >= 6)]
valid_normal = valid_normal['File'].tolist()

filtered_df_test_abnormal = df[(df['Class'] == 'abnormal') & (df['Type'] == 'test') & (df['Age'] >= 6)]

# Calculate the sum of SW and SSW columns
sw_sum_test_ab= filtered_df_test_abnormal['SW'].sum()
ssw_sum_test_ab = filtered_df_test_abnormal['SSW'].sum()

print('sw_sum_test_ab',sw_sum_test_ab)
print('ssw_sum_test_ab',ssw_sum_test_ab)

filtered_df_valid_abnormal = df[(df['Class'] == 'abnormal') & (df['Type'] == 'valid') & (df['Age'] >= 6)]

# Calculate the sum of SW and SSW columns
sw_sum_valid_ab= filtered_df_valid_abnormal['SW'].sum()
ssw_sum_valid_ab = filtered_df_valid_abnormal['SSW'].sum()


filtered_df_train_abnormal = df[(df['Class'] == 'abnormal') & (df['Type'] == 'train') & (df['Age'] >= 6)]

# Calculate the sum of SW and SSW columns
sw_sum_train_ab= filtered_df_train_abnormal['SW'].sum()
ssw_sum_train_ab = filtered_df_train_abnormal['SSW'].sum()
print(' ')
print('sw_sum_train_ab',sw_sum_train_ab + sw_sum_valid_ab)
print('ssw_sum_train_ab',ssw_sum_train_ab + ssw_sum_valid_ab)

print(' ')
print('sw_sum_valid_ab',(sw_sum_train_ab + sw_sum_valid_ab)*0.15)
print('ssw_sum_valid_ab',(ssw_sum_train_ab + ssw_sum_valid_ab)*0.15)


sw_sum_test_ab 22806.0
ssw_sum_test_ab 22371.0
 
sw_sum_train_ab 111723.0
ssw_sum_train_ab 74217.0
 
sw_sum_valid_ab 16758.45
ssw_sum_valid_ab 11132.55


In [4]:
print('Total Train Files: ',len(train_abnormal) ,len(train_normal), len(train_abnormal) + len(train_normal))
print('Total Test Files: ',len(test_abnormal) ,len(test_normal), len(test_abnormal) + len(test_normal))
print('Total Valid Files: ',len(valid_abnormal) ,len(valid_normal), len(valid_abnormal) + len(valid_normal))

Total Train Files:  57 547 604
Total Test Files:  18 168 186
Total Valid Files:  16 96 112


In [5]:
for waveletType in waveletsTypes:
    print(main_path + waveletType)
    if not os.path.exists(main_path + waveletType):
        os.makedirs(main_path + waveletType)

    for split in splits:
        if not os.path.exists(main_path + waveletType + '/' + split + '/' + dest_list[0]):
            os.makedirs(main_path + waveletType + '/' + split + '/' + dest_list[0])
            os.makedirs(main_path + waveletType + '/' + split + '/' + dest_list[1])
            os.makedirs(main_path + waveletType + '/' + split + '/' + dest_list[2])

/home/dll-1/Desktop/eeg/datasets/Adil paper/scallogramTest5/mexh
/home/dll-1/Desktop/eeg/datasets/Adil paper/scallogramTest5/morl
/home/dll-1/Desktop/eeg/datasets/Adil paper/scallogramTest5/gaus1
/home/dll-1/Desktop/eeg/datasets/Adil paper/scallogramTest5/gaus2


In [6]:
def imageExtraction(full_data,full_dataNormal,Epochs_data_normal,scales,main_path,split,dest_list,file_num,Label_data,Epochs_data,channelNum):
    ts = time.time()
    for waveletType in waveletsTypes:
        print('Working on Channel: ', channelNum,waveletType)
        i = channelNum
        window = []
        coef,_ = pywt.cwt(full_data[i], scales , waveletType, method = 'conv')
        coefNormal,_ = pywt.cwt(full_dataNormal[i], scales , waveletType, method = 'conv')
        for j in range(Epochs_data.shape[0]):
            if(not os.path.isfile(main_path + waveletType  + '/' + split + '/' + dest_list[1] + '/' + 'img_' + str(j) + '_' + str(i) + '_' + str(file_num) + '.png')):
                sig_cwt,_ = pywt.cwt(Epochs_data[j][i], scales , waveletType,method = 'conv')
                if Label_data[j][i] == 1:
                    # print(1)
                    # print(np.shape(sig_cwt))
                    plt.imshow(sig_cwt, extent=[1, 31, 31, 1], cmap='nipy_spectral', vmax=abs(coef).max(), vmin=-abs(coef).max())
                    plt.axis('off')
                    plt.savefig(fname = main_path + waveletType  + '/' + split + '/' + dest_list[1] + '/' + 'img_' + str(j) + '_' + str(i) + '_' + str(file_num) + '.png', bbox_inches = 'tight')
                    plt.clf()
                    plt.close()

                elif Label_data[j][i] == 2:
                    plt.imshow(sig_cwt, extent=[1, 31, 31, 1], cmap='nipy_spectral', vmax=abs(coef).max(), vmin=-abs(coef).max())
                    plt.axis('off')
                    plt.savefig(fname = main_path + waveletType  + '/' + split + '/' + dest_list[2] + '/' + 'img_' + str(j) + '_' + str(i) + '_' + str(file_num) + '.png', bbox_inches = 'tight')
                    plt.clf()
                    plt.close()
                
                else:
                    if j % 30 == 0:
                        sig_cwtNormal,_ = pywt.cwt(Epochs_data_normal[j][i], scales , waveletType,method = 'conv')
                        plt.imshow(sig_cwtNormal, extent=[1, 31, 31, 1], cmap='nipy_spectral', vmax=abs(coefNormal).max(), vmin=-abs(coefNormal).max())
                        plt.axis('off')
                        plt.savefig(fname = main_path + waveletType  + '/' + split + '/' + dest_list[0] + '/' + 'img_' + str(j) + '_' + str(i) + '_' + str(file_num) + '.png', bbox_inches = 'tight')
                        plt.clf()
                        plt.close()
    te = time.time()
    print('Tota Time Taken for Channel:',channelNum, te-ts)

In [7]:
def normal_image_extraction(full_data,scales,main_path,split,dest_list,file_num,Epochs_data,channelNum,win_ch):
    i = channelNum
    print("Compiling",i)
    for waveletType in waveletsTypes:
        coef,_ = pywt.cwt(full_data[i], scales,waveletType,method = 'conv')
        for j in range(win_ch):
            if(not os.path.isfile(main_path + waveletType  + '/' + split + '/' + dest_list[1] + '/' + 'img_' + str(j) + '_' + str(i) + '_' + str(file_num) + '.png')):
                rand_num = randint(0,Epochs_data.shape[0]-1)
                sig_cwt,_ = pywt.cwt(Epochs_data[rand_num][i], scales, waveletType, method = 'conv')
                plt.imshow(sig_cwt, extent=[1, 31, 31, 1], cmap='nipy_spectral', vmax=abs(coef).max(), vmin=-abs(coef).max())
                plt.axis('off')
                plt.savefig(fname = main_path + waveletType  + '/' + split + '/' + dest_list[0] + '/' + 'img_' + str(rand_num) + '_' + str(i) + '_' +  str(file_num) + '.png', bbox_inches = 'tight')
                plt.clf()
                plt.close()

In [8]:
if os.path.exists("processed_files_test.npz"):
    completed_files = np.load("processed_files_test.npz")['data'].tolist()
else:
    completed_files = []

print(completed_files)

for split in splits:
    if split == 'train':
        split_files_abnormal = train_abnormal
        split_files_normal = train_normal
    elif split == 'test':
        split_files_abnormal = test_abnormal
        split_files_normal = test_normal
    else:
        split_files_abnormal = valid_abnormal
        split_files_normal = valid_normal

    print('Split Type: ', split)
    print('Split Files :', split_files_abnormal)
    print('Split Files :', split_files_normal)
        
    for i,file in enumerate(split_files_abnormal):
        print("Processing:",i+1,"/",len(split_files_abnormal),file)
        if(file not in completed_files):
            file_num = int(file[:-4])
            edf_name = str(10000000 + file_num)[1:] + '.edf' 
            raw = mne.io.read_raw_edf(edfdir + "/" + edf_name,preload = True,exclude = ['A1','A2'],verbose=False)     # Importing all EEG Channels, exculding A1 A2 since matlab has already refrenced the channels with A1 and A2
            rawNormal = raw.copy()
            full_data = np.array(raw.get_data())
            epochs=mne.make_fixed_length_epochs(raw,duration=2,overlap=1,verbose=False)           #Setting overlapping duration of 1 second
            epochs_data=epochs.get_data(verbose=False)

            type,channels,beg,end = extractlabels(file_num,csvdir)
            type = cleanlabels(type)
            type, channels, beg, end = np.array(type), np.array(channels), np.array(beg), np.array(end)
            labels = np.array(generatelabelarray(type, channels))
            data = np.array(raw.get_data())
            label_data = np.transpose(sample_labeling(data,labels,beg,end))
            
            label_data = int_encoder(label_data)
            info_labels = mne.create_info(ch_names=['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'FZ', 'PZ', 'CZ'],sfreq = 200)   #Setting channels in MNE Object
            raw_labels = mne.io.RawArray(label_data,info_labels,verbose=False)                #Making MNE Raw object and making overlapping epochs of labels per sample
            epochs_labels=mne.make_fixed_length_epochs(raw_labels,duration=2,overlap=1,verbose=False)         #Setting overlapping duration of 1 second
            epochs_labels = epochs_labels.get_data(verbose=False)
            epochs_labels = epoch_windowing(epochs_labels) #(no of epochs, channels)
            Label_data = epochs_labels
            Epochs_data = epochs_data

            ############### Artifact removal For Normal Windows ###################
            rawNormal.filter(l_freq=1,h_freq=45,verbose=False)
            montage =  mne.channels.make_standard_montage('standard_1005')
            rawNormal.set_montage(montage, match_case=False,verbose=False)
            ica = mne.preprocessing.ICA(method="picard", max_iter="auto", random_state=56,verbose=False)
            ica.fit(rawNormal,verbose=False)
            muscle_idx_auto, scores = ica.find_bads_muscle(rawNormal,verbose=False)
            badIndexes = np.where(np.array(scores) > np.median(scores)*0.5)[0].tolist()
            ica.exclude = badIndexes
            print(f"Automatically found muscle artifact ICA components: {badIndexes}")
            ica.apply(rawNormal,verbose=False)
            full_dataNormal = np.array(rawNormal.get_data(verbose=False))
            epochs2=mne.make_fixed_length_epochs(rawNormal,duration=2,overlap=1,verbose=False)
            Epochs_data_normal=epochs2.get_data(verbose=False)
            #######################################################################

            num_processes = Epochs_data.shape[1]
            print("Epochs data shape AB",Epochs_data.shape)
            print("Epochs data shape NOR",Epochs_data_normal.shape)
            # You can provide a list of tuples as the second argument to map, where each tuple contains the input arguments
            if(Epochs_data.shape[0] < 4000):
                inputs = [(full_data,full_dataNormal,Epochs_data_normal,scales,main_path,split,dest_list,file_num,Label_data,Epochs_data,i) for i in range(num_processes)]
                with multiprocessing.Pool(processes=num_processes, maxtasksperchild=10) as pool:
                    pool.starmap(imageExtraction, inputs)
        
            else:    
                for i in range(num_processes):
                    imageExtraction(full_data,full_dataNormal,Epochs_data_normal,scales,main_path,split,dest_list,file_num,Label_data,Epochs_data,i)
                    # collected = gc.collect()

            completed_files = np.append(completed_files,file)
            np.savez("processed_files_test.npz",data=completed_files)

        collected = gc.collect()    
    collected = gc.collect()

    print('---------------------Going now Normal---------------------')
    random.seed(444)     #Make sure the seed is same to get similar results
    window_num = 195   #Define the number of windows you want per normal file
    '''
        This is done to downsample from the large amount of normal data present. This is done to mitigate the data imbalance
        You can get the total number of normal windows(images) produced = window_num * no of files. 
    '''
    win_ch = int(window_num/19)
    coef_data = np.empty((2,19))

    for i,file in enumerate(split_files_normal):
        print("Processing:",i+1,"/",len(split_files_normal),file)
        if(file not in completed_files):
            file_num = int(file[:-4])
            raw = mne.io.read_raw_edf(normalEdf_dir + file,preload = True,exclude = ['A1','A2'],verbose=False)     # Importing all EEG Channels, exculding A1 A2 since matlab has already refrenced the channels with A1 and A2
            raw.filter(l_freq=1,h_freq=45,fir_window='hamming',verbose=False)      # Bandpass filtering [2-45] Hz
            montage =  mne.channels.make_standard_montage('standard_1005')
            raw.set_montage(montage, match_case=False)
            ica = mne.preprocessing.ICA(method="picard", max_iter="auto", random_state=56,verbose=False)
            ica.fit(raw,verbose=False)
            muscle_idx_auto, scores = ica.find_bads_muscle(raw)
            badIndexes = np.where(np.array(scores) > np.median(scores)*0.5)[0].tolist()
            ica.exclude = badIndexes
            print(f"Automatically found muscle artifact ICA components: {badIndexes}")
            ica.apply(raw,verbose=False)
            full_data = np.array(raw.get_data(verbose=False))
            epochs=mne.make_fixed_length_epochs(raw,duration=2,overlap=0,verbose=False)
            epochs_data=epochs.get_data(verbose=False)  
            
            print('Shape of input data after Epochs:',epochs_data.shape)
            num_processes = 18
            inputs = [(full_data,scales,main_path,split,dest_list,file_num,epochs_data,i,win_ch) for i in range(num_processes)]
            
            with multiprocessing.Pool(processes=num_processes, maxtasksperchild=10) as pool:
                pool.starmap(normal_image_extraction, inputs)
            
            collected = gc.collect()
            completed_files = np.append(completed_files,file)
            np.savez("processed_files_test.npz",data=completed_files)

['0000952.edf', '0000502.edf', '0000538.edf', '0000004.edf', '0000908.edf', '0000219.edf', '0000676.edf', '0000507.edf', '0001041.edf', '0000281.edf', '0000585.edf', '0001066.edf', '0001005.edf', '0000764.edf', '0000599.edf', '0000919.edf', '0000827.edf', '0001064.edf']
Split Type:  train
Split Files : ['0000952.edf', '0000502.edf', '0000538.edf', '0000004.edf', '0000908.edf', '0000219.edf', '0000676.edf', '0000507.edf', '0001041.edf', '0000281.edf', '0000585.edf', '0001066.edf', '0001005.edf', '0000764.edf', '0000599.edf', '0000919.edf', '0000827.edf', '0001064.edf', '0000884.edf', '0000623.edf', '0001018.edf', '0000278.edf', '0000026.edf', '0000621.edf', '0000353.edf', '0000594.edf', '0000131.edf', '0000652.edf', '0000765.edf', '0000624.edf', '0000956.edf', '0001022.edf', '0001082.edf', '0000627.edf', '0000250.edf', '0000638.edf', '0000127.edf', '0001029.edf', '0000635.edf', '0000744.edf', '0001024.edf', '0000376.edf', '0000634.edf', '0000637.edf', '0000965.edf', '0000444.edf', '0000

In [ ]:
for waveType in waveletsTypes:
    for ab in dest_list:
        validFiles = os.listdir(main_path + waveType + '/' + 'valid' + "/" + ab)
        for f in validFiles:
            shutil.move(main_path + waveType + '/' + 'valid' + "/" + ab + '/' + f, main_path + waveType + '/' + 'train' + "/" + ab + '/' + f)

for waveType in waveletsTypes:
    for ab in dest_list:
        trainFiles = os.listdir(main_path + waveType + '/' + 'train' + "/" + ab + '/')
        files_to_move_valid = random.sample(trainFiles, int(len(trainFiles)*0.15))
        for f in files_to_move_valid:
            shutil.move(main_path + waveType + '/' + 'train' + "/" + ab + '/' + f, main_path + waveType + '/' + 'valid' + "/" + ab + '/' + f)

In [ ]:
f = np.load("processed_files_test.npz",allow_pickle=True)
len(f['data'].tolist())

1075

In [ ]:
len(f['data'].tolist())

51